In [9]:
import csv
import numpy as np
import time

file_path = 'files/filtered_feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        yield R
    while P:
        v = P.pop()
        yield from BronKerbosch1(
            P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v]))
        X.add(v)

# Executando o algoritmo de Bron–Kerbosch e medindo o tempo de execução
P = N.keys()
start_time = time.time()
cliques = list(BronKerbosch1(P))
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

# Salvando os resultados em um arquivo CSV
result_file_path = 'files/cliques_resultados.csv'
with open(result_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(cliques)

print(f"Resultados salvos em: {result_file_path}")


Tempo de execução: 0.1504814624786377 segundos
Resultados salvos em: files/cliques_resultados.csv


In [8]:
import csv
import numpy as np
import time

file_path = 'files/filtered_feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        yield R
    # Choose pivot vertex using the degree heuristic
    pivot_candidates = P.union(X)
    if pivot_candidates:
        pivot = max(pivot_candidates, key=lambda v: len(N[v]))
        for v in P - N[pivot]:
            yield from BronKerbosch1(
                P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v]))
            P.remove(v)
            X.add(v)

# Executando o algoritmo de Bron–Kerbosch e medindo o tempo de execução
P = set(N.keys())  # Convert keys to set
start_time = time.time()
cliques = list(BronKerbosch1(P))
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")


Tempo de execução: 0.0005533695220947266 segundos
Resultados salvos em: files/bron_kerbosch_results.csv
